## DIA 112: Subir Reportes a Firebase Storage con Protección por Token Único por Usuario


Hoy vas a implementar un sistema donde cada reporte generado se sube a Firebase Storage, protegido por un enlace único y seguro, accesible solo para el usuario correspondiente.

✅ ¿Qué aprenderás hoy?

🔐 Subir archivos a Firebase con pyrebase

📥 Generar URLs de descarga protegidas

🔑 Crear tokens únicos de acceso

🧾 Asociar enlaces al usuario

🧩 1. Instalar pyrebase
bash
Copiar
Editar
pip install pyrebase4
🧩 2. Configurar Firebase
Ve a https://console.firebase.google.com

Crea un nuevo proyecto.

Habilita Firebase Storage y selecciona modo de prueba o modo autenticado.

Descarga el archivo firebaseConfig.json (Web config).

🧩 3. Configuración de pyrebase en Python
python
Copiar
Editar
import pyrebase

firebase_config = {
  "apiKey": "TU_API_KEY",
  "authDomain": "tu-app.firebaseapp.com",
  "projectId": "tu-app",
  "storageBucket": "tu-app.appspot.com",
  "messagingSenderId": "XXX",
  "appId": "1:XXX:web:XXX",
  "measurementId": "G-XXX",
  "databaseURL": ""
}

firebase = pyrebase.initialize_app(firebase_config)
storage = firebase.storage()
🧩 4. Subir el archivo y obtener URL única
python
Copiar
Editar
import uuid

def subir_reporte_firebase(user, archivo_buffer, ext):
    nombre_archivo = f"reportes/{user.username}_{uuid.uuid4()}.{ext}"
    storage.child(nombre_archivo).put(archivo_buffer)
    url = storage.child(nombre_archivo).get_url(token=None)
    return url
El token en la URL es generado por Firebase automáticamente si el archivo no es público. Si quieres que solo el usuario vea su archivo, se recomienda usar autenticación personalizada o enviar por correo/telegram sin exponer en frontend.

🧩 5. Integrar con el sistema de generación de reportes
Después de generar el PDF/CSV y antes de enviar el correo o telegram:

python
Copiar
Editar
url_descarga = subir_reporte_firebase(user, archivo_buffer, tipo)
enviar_mensaje_con_qr(user.telegram_chat_id, url_descarga, user.username)
🧩 6. Seguridad y control de accesos
Firebase Storage usa reglas que puedes personalizar. Por ejemplo:

json
Copiar
Editar
service firebase.storage {
  match /b/{bucket}/o {
    match /reportes/{file} {
      allow read: if request.auth != null;
    }
  }
}
O también puedes hacer que expiren automáticamente borrándolos con un cron diario/semanal.